In [1]:
import pandas as pd 
import numpy as np



In [2]:
df = pd.read_csv("IMDB Dataset.csv")

In [4]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [5]:
df.shape

(50000, 2)

In [7]:
df['sentiment'].value_counts()

sentiment
positive    25000
negative    25000
Name: count, dtype: int64

In [10]:
df_encoded = df.replace({'sentiment' : {'positive': 1 , 'negative': 0}})

C:\Users\vinma\AppData\Local\Temp\ipykernel_21540\2558029091.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_encoded = df.replace({'sentiment' : {'positive': 1 , 'negative': 0}})


In [11]:
df_encoded.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [12]:
df_encoded['sentiment'].value_counts()

sentiment
1    25000
0    25000
Name: count, dtype: int64

In [ ]:
# for the following data we will use LSTM model (Long short-term memory) for sentiment analysis we will avoid using RNN as the data is textual 
#RNN mai memory nahi hai LSTM mai hai aur data mai vector zyada hai toh better rahega ki hamm LSTM use karree low data pai RnN chal  Jata

In [15]:
pip install tensorflow

  Using cached tensorflow-2.20.0rc0-cp313-cp313-win_amd64.whl.metadata (4.6 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl.metadata (4.4 kB)
  Using cached flatbuffers-25.2.10-py2.py3-none-any.whl.metadata (875 bytes)
  Using cached gast-0.6.0-py3-none-any.whl.metadata (1.3 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl.metadata (814 bytes)
  Using cached libclang-18.1.1-py2.py3-none-win_amd64.whl.metadata (5.3 kB)
  Using cached opt_einsum-3.4.0-py3-none-any.whl.metadata (6.3 kB)
  Using cached termcolor-3.1.0-py3-none-any.whl.metadata (6.4 kB)
  Using cached wrapt-1.17.2-cp313-cp313-win_amd64.whl.metadata (6.5 kB)
  Using cached tensorboard-2.20.0-py3-none-any.whl.metadata (1.8 kB)
  Using cached keras-3.11.1-py3-none-any.whl.metadata (5.9 kB)
  Using cached markdown-3.8.2-py3-none-any.whl.metadata (5.1 kB)
  Using cached tensorboard_data_server-0.7.2-py3-none-any.whl.metadata (1.1 kB)
Using cached tensorflow-2.20.0rc0-cp313-cp313-win_amd64.whl (332.0 MB)
Using cach


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [16]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM,Embedding 
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


In [ ]:
# here using pad_sequences for the first time it makes the token length by padding the shorter ones so that unifor tokens are provided to the nural network 
# aise provide karna is mandatory so tokenisation ke badd we will use padding 

In [19]:
train_data , test_data = train_test_split(df_encoded, test_size = 0.2, random_state=42)

In [18]:
train_data.shape

(40000, 2)

In [20]:
test_data.shape

(10000, 2)

In [21]:
tokenizer = Tokenizer(num_words =5000)
tokenizer.fit_on_texts(train_data['review'])

In [37]:
x_train = pad_sequences(tokenizer.texts_to_sequences(train_data['review']), maxlen=200)
x_test = pad_sequences(tokenizer.texts_to_sequences(test_data['review']), maxlen=200)

In [38]:
x_train

array([[1935,    1, 1200, ...,  205,  351, 3856],
       [   3, 1651,  595, ...,   89,  103,    9],
       [   0,    0,    0, ...,    2,  710,   62],
       ...,
       [   0,    0,    0, ..., 1641,    2,  603],
       [   0,    0,    0, ...,  245,  103,  125],
       [   0,    0,    0, ...,   70,   73, 2062]],
      shape=(40000, 200), dtype=int32)

In [39]:
x_test

array([[   0,    0,    0, ...,  995,  719,  155],
       [  12,  162,   59, ...,  380,    7,    7],
       [   0,    0,    0, ...,   50, 1088,   96],
       ...,
       [   0,    0,    0, ...,  125,  200, 3241],
       [   0,    0,    0, ..., 1066,    1, 2305],
       [   0,    0,    0, ...,    1,  332,   27]],
      shape=(10000, 200), dtype=int32)

In [46]:
y_train = train_data['sentiment']
y_test = test_data['sentiment']

In [45]:
y_train


39087    0
30893    0
45278    1
16398    0
13653    0
        ..
11284    1
44732    1
38158    0
860      1
15795    1
Name: sentiment, Length: 40000, dtype: int64

In [51]:
from tensorflow.keras.layers import Input

model = Sequential()
model.add(Input(shape=(200,)))
model.add(Embedding(input_dim = 5000, output_dim = 128))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

In [31]:
# input_dim = 5000 chee vocabulary size most frequent words ach consider karsee 
# output_dim = 128 chee embedding vector size je ki 128 dimensional vector banavsee
# input_length = 500 chee maximum length of the input sequence je ki 500 

In [29]:
# here droput drops 20% randomly from the input layer to avoid overfitting and recurrent_dropout drops 20% randomly from the output layer to avoid overfitting
# recurrent_droput mai bhi drop hi hOga just inside thw LSTM cell jo ki hidden connections hote hai
# 128 ji che ee lstm unit che je ki store karrse information anee decide karse waht to keep and wjat not , output apsee relevant to the next steps 

In [30]:
#apdee binary classification karre chee thus used 1 
#sigmoid activation pann use karre chee as binary classfification 
#model = Sequential()
# If output > 0.5 = class 1
# If output ≤ 0.5 = class 0

In [52]:
model.summary()

Model: "sequential_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_7 (Embedding)         │ (None, 200, 128)       │       640,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_7 (LSTM)                   │ (None, 128)            │       131,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 771,713 (2.94 MB)

 Trainable params: 771,713 (2.94 MB)

 Non-trainable params: 0 (0.00 B)

In [54]:
model.compile(optimizer= 'adam', loss='binary_crossentropy', metrics=['accuracy'])

In [53]:
# ayaa model.compile tells the model how to update weights , how to calculate error and what metrics to track 
#optimizer adam is algorithm to update weights works well for most of the cases also adjusts learning dynamically
#loss binary_crossentropy is used for binary classification problems measures the difference between predicted and actual values
#metrics accuracy is used to track the performance of the model during training and testing

In [55]:
model.fit(x_train, y_train, epochs = 5, batch_size=64, validation_split=0.2)

Epoch 1/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 128s 251ms/step - accuracy: 0.7856 - loss: 0.4613 - val_accuracy: 0.8581 - val_loss: 0.3388
Epoch 2/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 113s 225ms/step - accuracy: 0.8558 - loss: 0.3466 - val_accuracy: 0.8455 - val_loss: 0.3588
Epoch 3/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 109s 217ms/step - accuracy: 0.8749 - loss: 0.3028 - val_accuracy: 0.8608 - val_loss: 0.3319
Epoch 4/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 99s 197ms/step - accuracy: 0.8918 - loss: 0.2718 - val_accuracy: 0.8754 - val_loss: 0.3044
Epoch 5/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 98s 196ms/step - accuracy: 0.8897 - loss: 0.2684 - val_accuracy: 0.8529 - val_loss: 0.3574


In [56]:
loss , accuracy = model.evaluate(x_test, y_test)

313/313 ━━━━━━━━━━━━━━━━━━━━ 12s 36ms/step - accuracy: 0.8526 - loss: 0.3524


In [57]:
print(loss)

0.3523964285850525


In [60]:
print(accuracy)

0.8525999784469604


In [62]:
model.save('movie_review_model.keras')

In [63]:
import joblib
joblib.dump(tokenizer, 'tokenizer.pkl')

['tokenizer.pkl']

In [ ]:
def predictive_system(review):
    sequences = tokenizer.texts_to_sequences([review])
    
    padded_sequence = pad_sequences(sequences, maxlen=200)
    
    prediction = model.predict(padded_sequence)[0][0]
    
    if prediction >= 0.5:
        result = "Positive Review"
    else:
        result = "Negative Review"
    
    return f"Prediction Score: {prediction:.4f} → {result}"


In [69]:
print(predictive_system("I loved the movie, it was fantastic!"))


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 395ms/step
Prediction Score: 0.9099 → Positive 😃
